In [ ]:
!pip install -q rasterio

     |████████████████████████████████| 19.3 MB 1.2 MB/s 


In [ ]:
import numpy as np
import rasterio as rio
from rasterio.warp import reproject, Resampling, calculate_default_transform

In [ ]:
CRS = "EPSG:4326" 

# Lendo imagem original
with rio.open("/content/drive/MyDrive/Curso PDI com Python/L71221071_07120010720_DN.tif") as src:
    src_transform = src.transform

    # Matriz de transformação
    transform, width, height = calculate_default_transform(
        src.crs,
        CRS,
        src.width,
        src.height,
        *src.bounds,  # unpacks outer boundaries (left, bottom, right, top)
    )

    # Matadados do arquivo de saída
    meta = src.meta.copy()
    meta.update(
        {
            "crs": CRS,
            "transform": transform,
            "width": width,
            "height": height,
            "nodata": np.nan,  
        })

    # Reprojetando e gravando em disco
    with rio.open("L7_reproject.tif", "w", **meta) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=CRS,
                resampling=Resampling.nearest,
            )